<center><img src="../图片数据/logo.png" alt="Header" style="width: 800px;"/></center>

@Copyright (C): 2010-2019, Shenzhen Yahboom Tech  
@Author: Malloy.Yuan  
@Date: 2019-07-17 10:10:02  
@LastEditors: Malloy.Yuan  
@LastEditTime: 2019-09-17 17:54:19  

# 加载训练模型
我们通过运行'训练模型'把'采集数据'代码采集到的数据训练成了我们需要得到的避障模型之后,我们现在开始一步一步使用这个模型,实现避障的功能

执行以下代码，初始化PyTorch模型

In [1]:
import torch
import torchvision

model = torchvision.models.alexnet(pretrained=False)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)

接下来，加载您上传的，已经被训练过的``best_model.pth`` 的模型 

In [2]:
model.load_state_dict(torch.load('../collision_avoidance/best_model.pth'))

目前，模型权重计算位于CPU内存上，我们将模型通过我们前面课程跟大家介绍或的'CUDA'转移到GPU上执行,执行下面的代码以使用到GPU。

In [3]:
device = torch.device('cuda')
model = model.to(device)

# 预处理功能
现在我们加载了模型，但有一个小问题，就是我们的摄像头的图像格式要与训练模型时的图像格式完全相同。要做到这一点，我们需要做一些预处理。分如下几个步骤：

1. 从BGR转换为RGB模式
2. 从HWC布局转换为CHW布局
3. 使用与训练期间相同的参数进行标准化（我们的摄像机提供[0,255]范围内的值，并在[0,1]范围内训练加载的图像，因此我们需要缩放255.0
4. 将数据从CPU内存传输到GPU内存
5. 批量添加维度

In [4]:
import cv2
import numpy as np

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

我们现在定义了我们的预处理功能，可以将图像从相机格式转换为神经网络输入的格式。

现在，让我们显示我们的摄像头。 你现在应该对此非常熟悉。 我们还将创建一个滑块，用于显示机器人被阻挡的概率。

In [5]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)
#camera = Camera.instance(width=224, height=224, fps=10)
image = widgets.Image(format='jpeg', width=224, height=224)
blocked_slider = widgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(widgets.HBox([image, blocked_slider]))

创建驱动电机的robot实例.

In [6]:
from jetbot import Robot

robot = Robot()

接下来，我们创建一个函数，只要相机的值发生变化，就会调用该函数。 此功能将执行以下步骤

1. 预处理相机图像
2. 执行神经网络
3. 当神经网络输出表明我们被阻挡时，我们将向左转，否则我们继续前进。

In [7]:
import torch.nn.functional as F
import time

def update(change):
    global blocked_slider, robot
    x = change['new'] 
    x = preprocess(x)
    y = model(x)
    
    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y = F.softmax(y, dim=1)
    
    prob_blocked = float(y.flatten()[0])
    
    blocked_slider.value = prob_blocked
    
    if prob_blocked < 0.7:
        robot.forward(0.8)
    else:
        robot.left(0.7)
#     if prob_blocked < 0.6:
# #       robot.stop()
#          robot.forward(0.7)
#     else:
# #       robot.stop()
#          robot.left(0.7)
    
    #time.sleep(0.001)
        
update({'new': camera.value})  # we call the function once to intialize

我们已经创建了神经网络执行功能，但现在我们需要将它附加到相机进行处理。

我们用``observe`` 函数完成这个处理。

提示：此代码将开始移动机器人！ 请确保你的Jetbot位于可移动地带,避免跌落损坏!!!

In [8]:
camera.observe(update, names='value')  # this attaches the 'update' function to the 'value' traitlet of our camera

运行完上面单元格代码块后，Jetbot就开始为每个检测到的照片生成新命令。 首先将机器人放在地上，看看它遇到障碍物时的反应。

如果要停止此行为，可以通过执行以下代码来取消。

In [9]:
camera.unobserve(update, names='value')
time.sleep(1)
robot.stop()

也许你希望Jetbot在没有流式传输视频的情况下运行，这样会减少JetBot的运算负担。 执行如下代码，你可以取消摄像头的连接。
只是不推流到浏览器上，但在Jetbot上摄像头仍然是工作状态中的。

In [13]:
camera_link.unlink()  # don't stream to browser (will still run camera)

又如果要继续在浏览器显示视频，请执行以下代码。

In [14]:
camera_link.link()  # stream to browser (wont run camera)